In [1]:
import serial # This requires pyserial
import time
import numpy as np

In [2]:
# Open the serial port
ser = serial.Serial('/dev/ttyUSB0') # Linux system using USB to RS232 adapter
#ser = serial.Serial('COM3') # Windows system

# Set the correct communication parameters
ser.stopbits=2
ser.timeout = 3

# How long to wait (s) between sending and receiving query
WAIT_TIME = 0.1

# Simple Example

Here we read the model number of the connected device to test communication. 

In [3]:
# Send command to request model number
ser.write(b"*IDN?\r\n")

# Keep reading data until we encouter the newline character
data = b''
while True:
    charOut = ser.read()
    data = data + charOut
    if(charOut == b'\n'):
        break
        
print(data)

b'StanfordResearchSystems,DS345,38787,1.04\r\n'


# Base Communication Functions

Here we encapsulate the above procedure in a single function

In [4]:
def readData():
    data = b''
    while True:
        charOut = ser.read()
        data = data + charOut
        if(charOut == b'\n'):
            break
    return data.decode()

def write(command):
    comm = command + "\r\n"
    ser.write(comm.encode())
    
def query(command):
    write(command)
    time.sleep(WAIT_TIME)
    return readData()

In [5]:
# Ask for the model number
query("*IDN?")

'StanfordResearchSystems,DS345,38787,1.04\r\n'

# Useful Functions

In [6]:
# Frequency Stuff
#-------------------------
def read_freq():
    return float(query("FREQ?"))

def set_freq(freq):
    write("FREQ %.6f" % freq)
    
    
# Amplitude Stuff
#-------------------------
def read_amp(): #Give peak-to-peak amplitude
    result = query("AMPL?")
    result = result.replace('\r\n', '')
    out = result.replace('VR', '')
    out = out.replace('VP', '')
    out = float(out)
    if('VR' in result):
        out *= np.sqrt(2) # RMS Conversion to peak
    return out

def set_amp(val_Vpp):
    write("AMPL %.2fVP" % val_Vpp)
    
    
# Offset Stuff
#-------------------------
def read_offset():
    return float(query("OFFS?"))

def set_offset(offset_voltage):
    write("OFFS %.2f" % offset_voltage)
    
    
# Waveform Stuff
#-------------------------
wfm_table = ['SINE', 'SQUARE', 'TRIANGLE', 'RAMP', 'NOISE', 'ARB']
def read_waveform():
    res = query("FUNC?")
    res = int(res)
    return wfm_table[res]

def set_waveform(waveform_type):
    if(waveform_type not in wfm_table):
        print("Unknown waveform: %s" % waveform_type)
        return
    index = wfm_table.index(waveform_type)
    write("FUNC %d" % index)

# Set Parameters

In [7]:
set_waveform('SINE')

In [8]:
read_waveform()

'SINE'

In [9]:
set_offset(0.0)

In [10]:
read_offset()

0.0

In [11]:
set_freq(1e6)

In [12]:
read_freq()

1000000.0

In [13]:
set_amp(10.0)

In [14]:
read_amp()

10.0